In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.optimize import minimize 

In [2]:
stock_returns = pd.read_excel("../FERM_3_Assignment_1.xlsx", sheet_name="Daily_Returns_Q1_Q2")

In [3]:
stock_returns.head()

,date,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
0,2016-01-04,-0.000736,0.000976,0.014852,-0.018058,0.000000,0.022343,0.008006,-0.023029,-0.010178,...,0.006963,0.026925,0.013954,0.002899,-0.032335,-0.004554,0.020011,0.019501,-0.000378,0.001867
1,2016-01-05,-0.005018,-0.016069,-0.006321,-0.045479,-0.021533,-0.000951,-0.008006,0.018804,-0.005128,...,-0.015441,-0.062643,-0.011614,-0.012273,-0.056744,-0.010708,0.010534,-0.017713,-0.019281,-0.015985
2,2016-01-06,-0.017237,-0.043235,-0.026306,-0.031773,-0.036736,-0.023581,-0.031023,0.074229,-0.012937,...,-0.016280,-0.049700,-0.040530,-0.014546,0.009889,-0.023338,-0.017561,-0.028090,-0.028942,-0.030801
3,2016-01-07,-0.009301,-0.018113,-0.001967,-0.012678,-0.019545,-0.006596,0.000000,-0.018247,-0.007843,...,0.000895,0.037504,0.016888,-0.006605,0.011009,-0.025106,0.001719,-0.025754,-0.016807,-0.017752
4,2016-01-08,0.012082,0.004559,-0.012135,0.018175,0.007211,0.011696,-0.001660,-0.055982,0.002621,...,0.012150,-0.035384,-0.016888,0.014431,0.012092,0.019581,0.020731,-0.010268,0.010637,0.004963


In [4]:
mean_return_matrix = stock_returns.drop(columns=["date"]).mean()
mean_return_matrix

IBM     0.000241
GE     -0.001126
LLY     0.000038
F      -0.000223
BAC     0.001167
KR     -0.000807
ITUB    0.001658
GG      0.000147
SIRI    0.000608
ACB     0.005781
VALE    0.002696
PFE     0.000250
CTL    -0.000815
T       0.000247
ECA     0.001902
ABEV    0.000825
CVS    -0.000569
MRVL    0.001777
CSCO    0.000740
MO      0.000435
BMY    -0.000179
WFC     0.000273
HBAN    0.000614
dtype: float64

In [5]:
round(mean_return_matrix["GE"],4)

np.float64(-0.0011)

In [6]:
cov_matrix = stock_returns.drop(columns=["date"]).cov()
cov_matrix

,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,ACB,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
IBM,0.000127,0.000047,0.000019,0.000055,0.000070,0.000005,0.000073,0.000011,0.000032,0.000031,...,0.000024,0.000084,0.000046,0.000020,0.000058,0.000047,0.000016,0.000014,0.000050,0.000069
GE,0.000047,0.000139,0.000022,0.000070,0.000087,0.000015,0.000084,-0.000026,0.000039,-0.000014,...,0.000039,0.000085,0.000048,0.000039,0.000051,0.000041,0.000020,0.000032,0.000062,0.000076
LLY,0.000019,0.000022,0.000163,0.000030,0.000041,0.000020,0.000030,0.000011,0.000026,0.000011,...,0.000020,0.000019,0.000027,0.000029,0.000025,0.000034,0.000014,0.000071,0.000031,0.000030
F,0.000055,0.000070,0.000030,0.000204,0.000127,0.000035,0.000092,-0.000004,0.000071,0.000026,...,0.000032,0.000152,0.000051,0.000033,0.000089,0.000065,0.000011,0.000055,0.000094,0.000115
BAC,0.000070,0.000087,0.000041,0.000127,0.000296,0.000058,0.000129,-0.000121,0.000085,0.000044,...,0.000019,0.000214,0.000072,0.000045,0.000103,0.000069,-0.000009,0.000040,0.000175,0.000233
KR,0.000005,0.000015,0.000020,0.000035,0.000058,0.000366,-0.000016,-0.000024,0.000030,0.000042,...,0.000015,0.000049,0.000016,0.000063,0.000038,0.000026,0.000009,0.000028,0.000042,0.000043
ITUB,0.000073,0.000084,0.000030,0.000092,0.000129,-0.000016,0.000701,0.000130,0.000070,0.000099,...,0.000052,0.000294,0.000340,0.000038,0.000127,0.000113,0.000050,0.000063,0.000068,0.000106
GG,0.000011,-0.000026,0.000011,-0.000004,-0.000121,-0.000024,0.000130,0.000642,-0.000003,-0.000058,...,0.000030,0.000148,0.000090,-0.000015,0.000009,0.000014,0.000012,0.000016,-0.000079,-0.000110
SIRI,0.000032,0.000039,0.000026,0.000071,0.000085,0.000030,0.000070,-0.000003,0.000169,0.000052,...,0.000029,0.000094,0.000048,0.000020,0.000076,0.000047,0.000019,0.000018,0.000057,0.000063
ACB,0.000031,-0.000014,0.000011,0.000026,0.000044,0.000042,0.000099,-0.000058,0.000052,0.002376,...,-0.000006,0.000149,0.000103,0.000036,0.000066,0.000045,0.000019,-0.000018,0.000025,0.000057


In [7]:
round(cov_matrix.loc["IBM", "GE"], 6)

np.float64(4.7e-05)

In [8]:
def gmv_cvxpy(cov: np.ndarray, mu: np.ndarray, target_return : float,  bounds=None) -> np.ndarray:

    cov = np.asarray(cov, dtype=float)
    mu = np.asarray(mu, dtype=float).ravel()
    n = cov.shape[0]
    assert cov.shape == (n, n)
    assert mu.shape == (n,)

    w = cp.Variable(n)
    objective = cp.Minimize(cp.quad_form(w, cov))
    
    constraints = [
        cp.sum(w) == 1,
        mu @ w >= target_return,
    ]


    if bounds is not None:
        # bounds can be (lb, ub) scalars or arrays length n
        lb, ub = bounds
        constraints += [w >= lb, w <= ub]

    prob = cp.Problem(objective, constraints)
    # prob.solve(solver=cp.OSQP)  # SCS also works; OSQP is good for QPs
    prob.solve(solver="SCS", eps=1e-8, max_iters=200000)

    if w.value is None:
        raise RuntimeError("Optimization failed. Try a different solver or check covariance matrix.")

    return np.array(w.value).ravel()

In [9]:
targeted_return1 = 0.0005

w_port1 = gmv_cvxpy(cov_matrix, mean_return_matrix, targeted_return1)
print("portfolio weights:", w_port1)
print("portfolio mean return : ", round(np.sum(w_port1 * mean_return_matrix), 4))
print("vol %:", round(np.sqrt(w_port1.T @ cov_matrix @ w_port1)*100, 4))
print("variance:", round(w_port1.T @ cov_matrix @ w_port1, 6))

portfolio weights: [ 0.08902356 -0.02072524  0.02362748 -0.04343367  0.05847895  0.02239614
 -0.02499968  0.07048226  0.06518042  0.01991419 -0.02437887  0.17286329
 -0.035328    0.14679367  0.00437308  0.02232285  0.03227274  0.03501936
  0.06174382  0.23273382  0.04456108  0.03012468  0.01695407]
portfolio mean return :  0.0005
vol %: 0.5765
variance: 3.3e-05


In [10]:
round(np.sqrt(w_port1.T @ cov_matrix @ w_port1), 4)

np.float64(0.0058)

In [11]:
np.sqrt(w_port1.T @ cov_matrix @ w_port1)

np.float64(0.0057647371864474745)

In [12]:
def min_variance_given_return(V, mu, r_min=0.0005, x0=None, long_only=False):
    V = np.asarray(V, dtype=float)
    mu = np.asarray(mu, dtype=float)
    n = mu.size

    # Objective: x^T V x
    def fun(x):
        return x @ V @ x

    # Gradient helps a lot
    def jac(x):
        return 2.0 * (V @ x)

    # Equality: sum x = 1
    cons = [
        {"type": "eq", "fun": lambda x: np.sum(x) - 1.0,
         "jac": lambda x: np.ones_like(x)},
        # Inequality in SciPy is "fun(x) >= 0"
        {"type": "ineq", "fun": lambda x: (mu @ x) - r_min,
         "jac": lambda x: mu},
    ]

    bounds = None
    if long_only:
        bounds = [(0.0, 1.0)] * n  # common long-only choice; you can also use (0,None)

    if x0 is None:
        x0 = np.ones(n) / n

    res = minimize(fun, x0, jac=jac, method="SLSQP",
                   constraints=cons, bounds=bounds,
                   options={"ftol": 1e-12, "maxiter": 10_000, "disp": False})

    if not res.success:
        raise RuntimeError(f"Optimization failed: {res.message}")

    return res.x, res.fun, res


In [13]:
x_opt, var_opt, res = min_variance_given_return(cov_matrix, mean_return_matrix, r_min=0.0005, long_only=False)
round(var_opt,6)

np.float64(3.3e-05)

In [14]:
w_port1

array([ 0.08902356, -0.02072524,  0.02362748, -0.04343367,  0.05847895,
        0.02239614, -0.02499968,  0.07048226,  0.06518042,  0.01991419,
       -0.02437887,  0.17286329, -0.035328  ,  0.14679367,  0.00437308,
        0.02232285,  0.03227274,  0.03501936,  0.06174382,  0.23273382,
        0.04456108,  0.03012468,  0.01695407])

In [15]:
x_opt

array([ 0.08917668, -0.02083719,  0.02284052, -0.04303578,  0.05731253,
        0.02201726, -0.02518617,  0.07055633,  0.06539291,  0.01996537,
       -0.02433193,  0.17212229, -0.03507993,  0.14724747,  0.00426807,
        0.02246318,  0.03250857,  0.03563451,  0.06099551,  0.23302198,
        0.04377781,  0.03257602,  0.016594  ])

In [16]:
print("portfolio mean return: ", round(np.sum(x_opt * mean_return_matrix)*100, 4))

portfolio mean return:  0.05


## targeted return should be greater or equal to 8%

In [17]:
targeted_return2 = 0.0008

w_port2 = gmv_cvxpy(cov_matrix, mean_return_matrix, targeted_return2)
print("portfolio weights:", w_port2)
print("portfolio mean return : ", round(np.sum(w_port2 * mean_return_matrix), 4))
print("vol:", round(np.sqrt(w_port2.T @ cov_matrix @ w_port2)*100, 6))
print("variance:", round(w_port2.T @ cov_matrix @ w_port2, 6))

portfolio weights: [ 0.08492824 -0.0896047   0.01297462 -0.06553585  0.10546914  0.00946691
 -0.02114152  0.06952578  0.06759035  0.02872426 -0.01578853  0.19202727
 -0.05050355  0.17399892  0.00620653  0.01368308  0.0216848   0.05155178
  0.08327695  0.25651009  0.04423664  0.01252062  0.00819819]
portfolio mean return :  0.0008
vol: 0.615127
variance: 3.8e-05


## targeted return should be greater or equal to 0.1%

In [18]:
targeted_return3 = 0.001

w_port3 = gmv_cvxpy(cov_matrix, mean_return_matrix, targeted_return3)
print("portfolio weights:", w_port3)
print("portfolio mean return : ", round(np.sum(w_port3 * mean_return_matrix), 4))
print("vol:", round(np.sqrt(w_port3.T @ cov_matrix @ w_port3)*100, 6))
print("variance:", round(w_port3.T @ cov_matrix @ w_port3, 8))

portfolio weights: [ 0.08219804 -0.13552435  0.00587271 -0.08027064  0.13679592  0.00084743
 -0.01856942  0.06888813  0.06919697  0.03459764 -0.01006164  0.20480326
 -0.06062059  0.19213574  0.00742883  0.00792324  0.01462617  0.06257339
  0.09763237  0.27236094  0.04402034  0.00078458  0.00236094]
portfolio mean return :  0.001
vol: 0.651047
variance: 4.239e-05


In [19]:
x1 = 3.3e-05
x2 = 3.8e-05

phi = (0.08 - 0.1) / (0.08 - 0.05) 

In [20]:
round(phi * x1 + (1-phi)*x2,8)

4.133e-05

In [21]:
phi_range = np.arange(-2, 2, 0.0001)

In [22]:
z_range = phi_range * x1 + (1-phi_range)*x2

In [23]:
z_range

array([4.80000e-05, 4.79995e-05, 4.79990e-05, ..., 2.80015e-05,
       2.80010e-05, 2.80005e-05], shape=(40000,))

## Sharpe minimization

In [24]:
def sharpe_via_unit_excess_return(cov, mu, rf, long_only=False):
    cov = np.asarray(cov, float)
    cov = 0.5 * (cov + cov.T)
    mu = np.asarray(mu, float).ravel()
    n = len(mu)

    x = cp.Variable(n)
    excess = mu - rf * np.ones(n)

    objective = cp.Minimize(cp.quad_form(x, cov))
    constraints = [excess @ x == 1]     
    if long_only:
        constraints += [x >= 0]

    prob = cp.Problem(objective, constraints)
    # prob.solve(solver=solver)
    prob.solve(solver="SCS", eps=1e-8, max_iters=500000)

    if x.value is None:
        raise RuntimeError(f"Optimization failed: {prob.status}")

    x_raw = np.array(x.value).ravel()

    # Convert to actual portfolio weights (fully invested): 
    w_star = x_raw / x_raw.sum()
    return w_star, x_raw

In [25]:
sharpe_return = 0.0001

In [26]:
w_sharp, x_raw  = sharpe_via_unit_excess_return(cov_matrix, mean_return_matrix, sharpe_return,
                                                long_only=False)
w_sharp

array([ -0.85485566, -15.89592109,  -2.43162209,  -5.13749855,
        10.88867844,  -2.95750452,   0.86421984,  -0.14996536,
         0.62061604,   2.05044082,   1.95550469,   4.58973717,
        -3.53295879,   6.41700122,   0.42694347,  -1.96895297,
        -2.40801367,   3.84537706,   5.02465461,   5.71264006,
        -0.03021543,  -4.02722283,  -2.00108246])

In [27]:
mu = np.asarray(mean_return_matrix).ravel()
w = np.asarray(w_sharp).ravel()
rf = sharpe_return  # risk-free rate in same periodicity as mu

excess_return = w @ mu - rf
vol = np.sqrt(w @ cov_matrix @ w)

sharpe_ratio = excess_return / vol

round(sharpe_ratio,3)

np.float64(0.26)

In [28]:
sharpe_ratio

np.float64(0.2602240442489255)

In [29]:
def sharpe_direction_closed_form(V, mu, rf):
    V = np.asarray(V, float)
    mu = np.asarray(mu, float)
    n = mu.size
    ones = np.ones(n)
    a = mu - rf * ones

    V = 0.5 * (V + V.T)

    # Solve V y = a  (more stable than explicitly inverting V)
    y = np.linalg.solve(V, a)
    x_star = y / (a @ y)

    w_star = x_star / (ones @ x_star)
    return x_star, w_star

In [30]:
x_star, w_star = sharpe_direction_closed_form(cov_matrix, mean_return_matrix, sharpe_return)
w_star

array([ -0.85485566, -15.89592109,  -2.43162209,  -5.13749855,
        10.88867844,  -2.95750452,   0.86421984,  -0.14996536,
         0.62061604,   2.05044082,   1.95550469,   4.58973717,
        -3.53295879,   6.41700122,   0.42694347,  -1.96895297,
        -2.40801367,   3.84537706,   5.02465461,   5.71264006,
        -0.03021543,  -4.02722283,  -2.00108246])

In [31]:
w_sharp

array([ -0.85485566, -15.89592109,  -2.43162209,  -5.13749855,
        10.88867844,  -2.95750452,   0.86421984,  -0.14996536,
         0.62061604,   2.05044082,   1.95550469,   4.58973717,
        -3.53295879,   6.41700122,   0.42694347,  -1.96895297,
        -2.40801367,   3.84537706,   5.02465461,   5.71264006,
        -0.03021543,  -4.02722283,  -2.00108246])